In [13]:
import h5py as h5
import tensorflow as tf
import os
import sys
sys.path.append('./scripts')
from nn import nn2
from train import train_model, make_train_figs

# GPU on
gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    print(gpu)
    tf.config.experimental.set_memory_growth(gpu, True)

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [14]:
Shape = (100,3)
model = nn2(Shape, is_mask = True)

(None, 25) (None, 25, 64)


In [15]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 100, 3)]     0           []                               
                                                                                                  
 conv1d_2 (Conv1D)              (None, 100, 128)     12416       ['input_2[0][0]']                
                                                                                                  
 p_re_lu_2 (PReLU)              (None, 100, 128)     128         ['conv1d_2[0][0]']               
                                                                                                  
 tf.__operators__.getitem_2 (Sl  (None, 100, 1)      0           ['input_2[0][0]']                
 icingOpLambda)                                                                             

In [16]:
import numpy as np
from nn_constructor import *
b = bidir_class(64, True, act = 'tanh', rec_act = 'sigmoid', merge_mode = 'mul')
x = np.random.uniform(size = (2,5,3))
mask = np.expand_dims(np.array([[1,1,1,1,0],[1,1,1,0,0]]), axis = -1)
print(mask.shape,x.shape)
print(b.block(x, mask_lstm = tf.cast(mask, bool))*mask)

(2, 5, 1) (2, 5, 3)
(2, 5, 1) (2, 5, 3)
tf.Tensor(
[[[ 4.31624358e-04  1.14983774e-03  1.36700657e-03  3.30452994e-03
    1.88302263e-04 -5.93915058e-04 -9.34389609e-05 -6.59035053e-04
   -3.78206227e-04  5.76371152e-04  3.86828469e-04  5.49790217e-04
   -2.90619140e-03  1.32648228e-03 -1.28905894e-03  3.94236937e-04
    2.40742695e-03 -2.81129760e-04 -1.62886386e-03 -2.89454794e-04
    1.77813548e-04  1.68447004e-04  9.95579394e-05 -4.39112727e-03
    4.70328559e-05 -4.71946078e-05  8.90783151e-04 -1.55128902e-04
    3.34389129e-04  2.30547297e-03 -5.55412436e-04  1.08611025e-03
   -1.94954197e-03  1.00053090e-03 -8.41965884e-05 -1.74288312e-03
   -2.59779254e-03 -9.81485471e-04 -1.24209240e-04 -9.52250339e-05
   -1.02406763e-03 -1.50901615e-03 -1.17131078e-03  3.39742110e-04
   -7.95863976e-04 -3.77641176e-04 -1.37924962e-03 -8.72826087e-04
   -5.39326167e-04 -2.39818532e-04 -6.15702651e-04 -4.24558093e-04
   -5.89544594e-04  4.72704269e-04  4.76979039e-04  3.33207194e-04
    1.22970

In [6]:
mask[:,:,0].shape

(2, 5)

In [2]:
import sys
sys.path.append('./scripts')
from ds_making import *

is_mask = True
batch_size = 256
file_name = "baikal_mu-nu_h5-s2_tres08_old_norm.h5"
path_to_h5 = '../../../../ivkhar/Baikal/data/' + file_name

with h5.File(path_to_h5, 'r') as hf:
        total_num = hf['train/data/data'].shape[0]
        steps_per_epoch = (total_num // batch_size) // 1
        Shape = hf['train/data/data'].shape[1:]
        #Учёт маски
        if is_mask:
            Shape = list(Shape)
            Shape[1] += 1
            Shape = tuple(Shape)


train_dataset = make_train_dataset(path_to_h5, batch_size, Shape, is_mask = is_mask)

2023-05-03 13:32:00.155265: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-03 13:32:03.761985: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 11209 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:44:00.0, compute capability: 8.6


<BatchDataset element_spec=(TensorSpec(shape=(None, 256, 96, 6), dtype=tf.float32, name=None), TensorSpec(shape=(None, 256, 2), dtype=tf.float32, name=None), TensorSpec(shape=(None, 256), dtype=tf.float32, name=None))>


In [75]:
import sys
import h5py as h5
sys.path.append('./scripts')
from ds_making import make_train_dataset, make_test_dataset, make_datasets_val

In [76]:
file_name = "baikal_mu-nu_h5-s2_tres08_old_norm.h5"
path_to_h5 = '../../../../ivkhar/Baikal/data/' + file_name
batch_size = 256
### getting the shape of data
with h5.File(path_to_h5, 'r') as hf:
    Shape = hf['test/data/data'].shape[1:]
    Shape = tuple([None,Shape[1]+1])
    pure_data = hf['test/data/data'][0:256]
test_dataset = make_test_dataset(path_to_h5, batch_size, Shape, is_mask = True)

In [77]:
el = next(test_dataset.as_numpy_iterator())

In [87]:
delta = el[0][0:10,:,:-1]-pure_data[0:10]
n = 2
c = 3
el[0][n,0:10,c], pure_data[n,0:10,c], delta[n,0:10,c]

(array([ 1.5174608 ,  1.5073937 ,  1.0566024 ,  1.0578704 ,  1.0630524 ,
         1.0621067 , -0.12455566, -0.1331442 ,  3.        ,  3.        ],
       dtype=float32),
 array([ 1.5139594,  1.5139594,  1.0626556,  1.0626556,  1.0626556,
         1.0626556, -0.1293371, -0.1293371,  3.       ,  3.       ],
       dtype=float32),
 array([ 0.00350142, -0.00656569, -0.00605321, -0.00478518,  0.00039685,
        -0.00054884,  0.00478144, -0.0038071 ,  0.        ,  0.        ],
       dtype=float32))

In [1]:
from scripts.losses import *

2023-09-05 20:38:00.931910: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-05 20:38:01.889358: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/albert/miniconda3/envs/Baikal2/lib/
2023-09-05 20:38:01.889446: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/albert/miniconda3/envs/Baikal2/lib/
2023-09-05 20:38:01.889456: W tensorflow/com

In [2]:
import re
str = '3 товара за 200.99'
pat = r'\d+.\d'
match = re.search(pat,str)
print(match.group())

200.9
